# テストの実行

In [77]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

import optuna
from ipywidgets import Play, IntSlider, jslink, HBox, interactive_output
from matplotlib import pyplot as plt


In [78]:
TESTSET_DIR = os.path.join('/home', 'jovyan', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jovyan', 'work', '03_opt')
PROG_PATH = os.path.join(PRJ_DIR, 'main')

## マスタの読み込み

In [79]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_testset_pre_master.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_testset_sys_master.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_testset_stress_master.csv'), usecols=['seed'])

In [80]:
CHAMP_TAG = 'yyyymmdd_hhmm'
CHAMP_DIR = os.path.join('/home', 'jovyan', 'work', 'result', 'champion')

champ_path = os.path.join(CHAMP_DIR, 'champ_all_{}.csv'.format(CHAMP_TAG))
champ_df = pd.read_csv(champ_path)

champ_score_dict = {}

for _, row in champ_df.iterrows():
    seed = row['seed']
    score = row['champion_score']

    champ_score_dict[seed] = score

## 実行するロジックの指定

In [81]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'prog-tag'

In [82]:
def solve(seed, param_list):
    problem_path = os.path.join(TESTSET_DIR, 'in', '{:0>4}.txt'.format(seed)) 
    param_str = ' '.join(map(str, param_list))
    command_str = 'echo {} {} | {}'.format(problem_path, param_str, PROG_PATH)

    # stack overflow対策
    # command_str = 'ulimit -S -s 1048576 && echo {} | {}'.format(problem_path, PROG_PATH)

    start_time = time.perf_counter()
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    
    #print('{}'.format(prob_id))    
    return (seed, e_time, res)

In [83]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name + '_master.csv')
    testset_df = pd.read_csv(testset_path)

    # 24並列実行
    with futures.ThreadPoolExecutor(max_workers=24) as executor:
        seed_list = testset_df['seed'].to_list()
        future_list = list(tqdm(executor.map(solve, seed_list), total=len(seed_list)))

    for future in future_list:
        seed, e_time, res = future

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # -- start -- 生成コード貼り付け先
            elem_cnt = 2

            result = str(res.stderr.decode('utf-8').split()[-elem_cnt + 0].replace('Result=', ''))
            score = int(res.stderr.decode('utf-8').split()[-elem_cnt + 1].replace('Score=', ''))

            solve_result.append(score)
            solve_result.append(result)   
            # -- end -- 生成コード貼り付け先

            # 相対スコア
            # rel_score = int(10 ** 9 * top_rate * champ_score_dict[seed] / score)
            # solve_result.append(rel_score)

        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    cols = ['testset', 'seed', 'time', 'score', 'result']
    result_df.columns = cols

    return result_df

In [84]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [85]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            # -- start -- 生成コード貼り付け先
            'score_mean': [np.mean(test_result_df['score'])],
            'score_min': [min(test_result_df['score'])],
            'score_max': [max(test_result_df['score'])],
            # -- end -- 生成コード貼り付け先

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    
    cols = ['tag']
    cols.extend(summary_df.columns)
    
    summary_all_df = summary_all_df[cols]
    
    return summary_all_df

In [59]:
#testset_name = '01_testset_pre'
#testset_name = '02_testset_sys'
#testset_name = '03_testset_stress'
testset_name = '04_testset_param'

result_dict = {}
summary_all_dict = {}

result_df = pd.DataFrame()

init_param_list = []

testset_result_df = run_test(testset_name, init_param_list)
result_df = pd.concat([result_df, testset_result_df], axis=0)

result_dict[PROG_NAME] = result_df
summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-03-30 15:46:03,187: Start
2023-03-30 15:46:15,029: finish!


In [60]:
sum(testset_result_df['score'])

538

In [67]:
def calc_score(param_list):
    testset_name = '04_testset_param'
    testset_result_df = run_test(testset_name, param_list)
    
    score = np.mean(testset_result_df['log_score'])

    return score

In [70]:
def objective(trial):
    target_double_up_turn = trial.suggest_int('target_double_up_turn', 840, 900) 
    
    double_up_cost_ub0 = trial.suggest_int('double_up_cost_ub0', 880, 950) 
    double_up_cost_ub1 = trial.suggest_int('double_up_cost_ub1', 430, 500) 
    double_up_cost_ub2 = trial.suggest_int('double_up_cost_ub2', 350, 450) 
    double_up_cost_ub3 = trial.suggest_int('double_up_cost_ub3', 270, 340) 
    double_up_cost_ub4 = trial.suggest_int('double_up_cost_ub4', 580, 700) 

    stack_money_lb = trial.suggest_int('stack_money_lb', 370, 460) 
    stack_turn = trial.suggest_int('stack_turn', 620, 720) 

    param_list = []

    param_list.append(target_double_up_turn)

    param_list.append(double_up_cost_ub0)
    param_list.append(double_up_cost_ub1)
    param_list.append(double_up_cost_ub2)
    param_list.append(double_up_cost_ub3)
    param_list.append(double_up_cost_ub4)
    
    param_list.append(stack_money_lb)
    param_list.append(stack_turn)
    
    return calc_score(param_list) 

In [75]:
study = optuna.create_study(direction="maximize")

study.enqueue_trial({'target_double_up_turn': init_param_list[0],
                    'double_up_cost_ub0': init_param_list[1],
                    'double_up_cost_ub1': init_param_list[2],
                    'double_up_cost_ub2': init_param_list[3],
                    'double_up_cost_ub3': init_param_list[4],
                    'double_up_cost_ub4': init_param_list[5],
                     
                    'stack_money_lb': init_param_list[6],
                    'stack_turn': init_param_list[7],
                    })

study.optimize(objective, timeout=10)

[I 2023-03-30 16:05:06,867] A new study created in memory with name: no-name-104bcd88-f1a7-4cd6-a818-2336e91665e1
2023-03-30 16:05:06,868: Start
2023-03-30 16:05:28,749: finish!
[I 2023-03-30 16:05:28,750] Trial 0 finished with value: 1834.0 and parameters: {'invalid_shift': 68, 'invalid_clear': 1}. Best is trial 0 with value: 1834.0.
2023-03-30 16:05:28,751: Start
2023-03-30 16:05:49,202: finish!
[I 2023-03-30 16:05:49,204] Trial 1 finished with value: 2043.0 and parameters: {'invalid_shift': 97, 'invalid_clear': 13}. Best is trial 0 with value: 1834.0.
2023-03-30 16:05:49,205: Start
2023-03-30 16:06:10,889: finish!
[I 2023-03-30 16:06:10,890] Trial 2 finished with value: 1969.0 and parameters: {'invalid_shift': 65, 'invalid_clear': 12}. Best is trial 0 with value: 1834.0.
2023-03-30 16:06:10,891: Start
2023-03-30 16:06:30,843: finish!
[I 2023-03-30 16:06:30,844] Trial 3 finished with value: 2047.0 and parameters: {'invalid_shift': 94, 'invalid_clear': 3}. Best is trial 0 with value: 

In [76]:
study.best_params

{'invalid_shift': 68, 'invalid_clear': 1}

In [74]:
int(study.best_value)

1944